In [1]:
#import libraries
import numpy as np
import matplotlib as plt
import openpyxl as openpy
import xlrd as xlrd
#Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
#API and geocoding libraries
import googlemaps
import gmplot
#Library to generate an HTML file
import webbrowser
#Library that can implement a delay before or after a call
from time import sleep
#Map visualization libraries
import folium
from folium.plugins import Draw
import geopandas as gpd
import branca
#Library that allows viewing of file in Jupyter
from IPython.display import display, IFrame
import base64
#Library that allows for dropdown filters(not functioning as this is an executable library)
#from streamlit_folium import st_folium

In [2]:
df = pd.read_excel(r"YOUR DATA SET")

In [3]:
#convert custom images using base64 to work with HTML elements
with open("water.png", "rb") as f:
    data = f.read()
    encoded = base64.b64encode(data).decode("utf-8")

In [7]:
df['col1'].unique()

array(['Business Programs', 'Community Facilities',
       'Single Family Housing', 'Water and Environmental'], dtype=object)

In [8]:
df=df.replace(to_replace="col1", value="new col1")

In [9]:
df['col1'].unique()

array(['Business Programs', 'Community Facilities',
       'Single Family Housing', 'Water and Environmental'], dtype=object)

In [10]:
#shape file for single state filter
# shape_file = gpd.read_file(r"location of shape file")
# SDC = shape_file[shape_file["STATEFP"]=="46"]
# SDC.to_file('South Dakota Counties.geojson', driver = "GeoJson")

In [11]:
#shape file for multi state filter
#shape_file = gpd.read_file(r"location of shape file")
# filter1 = shape_file["STATEFP"] == "38"
# filter2 = shape_file["STATEFP"] == "46"
# combined_filter = filter1 & filter2
# filter_gdf = shape_file[combined_filter]
# filter_gdf.to_file('filter_gdf.geojson', driver = "GeoJson")

In [12]:
#Create a DF from the GeoJson file for county outlines from their geometry
counties = gpd.read_file("Name of shape file.geojson")

In [13]:
#create a dropdown filter (not functioning)
#selected_city = st.selectbox("Choose a Program", df["Program"].unique())

In [14]:
column_headers = df.columns.tolist()
print(column_headers)

['Program', 'Sub-Program', 'State', 'Fiscal Year', 'County', 'Project Name', 'Project Description', 'latitudes', 'longitudes', 'Obligation Amount']


In [15]:
# Convert the entire 'latitudes' and 'longitudes' columns to numeric,
# coercing errors to NaN
df['latitudes'] = pd.to_numeric(df['latitudes'], errors='coerce')
df['longitudes'] = pd.to_numeric(df['longitudes'], errors='coerce')

# Combine the columns into a tuple, filtering out rows with missing values if needed
df = df.dropna(subset=['latitudes', 'longitudes'])
df['coordinates'] = list(zip(df['latitudes'], df['longitudes']))


In [16]:
#Create a base map
fmap = folium.Map(
    location=[lat, lon], 
    control_scale=True, 
    zoom_start=7, 
    tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 
    attr='&copy; OpenStreetMap contributors'
)
#47.54 -100.437012 for ND
#Ability to draw lines on the map
draw = Draw(
    draw_options={
        'polyline': {
            'shapeOptions': {
                'color': 'blue',
                'weight': 8,  # Thickness of line
                'opacity': 0.7
            }
        },
        'rectangle': False,  # Hide rectangle tool
        'circle': False,
        'marker': False,
        'polygon': False,
        'circlemarker': False
    },
    edit_options={'edit': True}
)

# Add Draw to map
draw.add_to(fmap)


#Outline counties
folium.GeoJson(counties, name="counties", style_function=lambda feature: {
    "fillColor": "#16254C",
    "color": "#26254C",
    "weight": 1,
    "fillOpacity": 0.1,
}).add_to(fmap)
#Opacity think in revese. 0.9 is only 10% opaque
            

# #Define colors for each category
# program_icon_color = {
#     'col1': 'green',
#     'col2': 'orange',
#     'col3': 'darkblue',
#     'col4': 'red'
# }



# #Define Icons for each category
# program_icon_name = {
#     'col1': 'shop',
#     'col 2': 'hospital',
#     'col 3': 'water',
#     'col 4': 'house'
# }
   
    
# For each Program, create a separate FeatureGroup
for program in df['Program'].unique():
    fg = folium.FeatureGroup(name=program) 
    subset = df[df['Program'] == program]
    
#Plot the points    
    for _, row in subset.iterrows():
        latlon = (row['latitudes'], row['longitudes'])
        pname = row['Project Name']
        
     
        pop_text = (
            f"<b>Project Description:</b> {row['col1']}<br>"
            f"<b>Program:</b> {program}<br>"
            f"<b>County:</b> {row['col 2']}<br>"
            f"<b>Fiscal Year:</b> {row['col 3']}"
        )
    
        if program == 'col entry 1':
            icon = folium.CustomIcon('custom icon 1.png', icon_size=(32,32))
        elif program == 'col entry 2':
            icon = folium.CustomIcon('custom icon 2.png', icon_size=(32,32))
        elif program == 'col entry 3':
            icon = folium.CustomIcon('custom icon 3.png', icon_size=(32,32))
        elif program == 'col entry 4':
            icon = folium.CustomIcon('custom icon 4.png', icon_size=(32,32))
            
            
        folium.Marker(
            location=latlon,
            tooltip=pname,
            popup=folium.Popup(pop_text, max_width=300),
            icon=icon
            ).add_to(fg)

        fg.add_to(fmap) 


#Create a feature group
folium.LayerControl(collapsed=True).add_to(fmap)
              

legend_html = f"""
{{% macro html(this, kwargs) %}}
<div style="
    position: fixed; 
    bottom: 0px; left: 0px; width: 195px; height: 120px; 
    border:2px solid grey; z-index:9999; font-size:14px;
    background-color:white; opacity: 0.9;
    padding: 10px;">
<b>Legend</b><br>
<img src="data:image/png;base64,{encoded}" style="height:20px;"> custom icon 1<br>
<img src="data:image/png;base64,{encoded2}" style="height:20px;"> custom icon 2<br>
<img src="data:image/png;base64,{encoded3}" style="height:20px;"> custom icon 3<br>
<img src="data:image/png;base64,{encoded4}" style="height:20px;"> custom icon 4<br>
</div>
{{% endmacro %}}
"""


legend = branca.element.MacroElement()
legend._template = branca.element.Template(legend_html)

fmap.get_root().add_child(legend)
    
#Print the map  
fmap.save('name of file.html')

#Display map in Juypter
display(IFrame('name of file.html', width='100%', height='500px'))